<a href="https://colab.research.google.com/github/davidrtorres/dsc-mod-4-project-v2-1-onl01-dtsc-pt-041320/blob/master/ts_model_notebook_12_15_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model 4 Time Series Notebook**

### **Introduction**
Business Problem: I am a consultant for Premium Real Estate, LLC.  The firm asked me to provide analysis and recommendations for investing in the top 5 zipcodes in Brooklyn that will provide the highest return on investment.  The investment firm is looking for short-term investments with the highest returns over a 3 year period.  The investment firm isn't looking for long term investments, ie, where a family is looking to purchase a home and stay in the home 10 to 15 years to build equity.<br>
<br>
I will make recommendations based on the market forecasting of real estate prices in Brooklyn. The top 5 zipcodes or 'best' zipcodes will be those with the highest ROI over the 3 year period.<br>
<br>
For the task, I analyzed real estate sales data from Zillow which covers time period 4-1-1996 to 4-1-2018.<br>
I used an auto_arima model to conduct a gridsearch and find the lowest AIC scores and corresponding p,d,qs and Seasonal P,D,Qs.  I used SARIMA model to make predictions regarding the test data so I could get an idea of how my models were working with making predictions.  I used RMSE to evaluate how my models were performing.  I then made models to perform dynamic forecasts for 3 years.<br>


In [1]:
print('Colab Notebook 12-15-20')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings('ignore')
import itertools
import statsmodels.api as sm

#from matplotlib.pylab import rcParams

Colab Notebook 12-15-20


/Users/davidtorres/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## **Auto_Arima Model**
The auto-ARIMA process seeks to identify the most optimal parameters for an ARIMA model.  In the ARIMA model there are 3 parameters that are used to help model the major aspects of a times series: 
  seasonality, trend, and noise. These parameters are labeled p,d,and q.<br>
 
p is the parameter associated with the auto-regressive aspect of the model, which incorporates past values. 
  For example, forecasting that if it rained a lot over the past few days, you state its likely that it will rain tomorrow as well.<br>  
d is the parameter associated with the integrated part of the model, which effects the amount of 
differencing to apply to a time series. ie, forecasting that the amount of rain tomorrow will be similar to the amount of rain today, if the daily amounts of rain have been similar over the past few days.<br>

q is the parameter associated with the moving average part of the model.<br>

P,D, and Q describe the same associations as p,d, and q, but correspond with the seasonal components 
  of the model.<br>

In [33]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [34]:
!pip install pmdarima

In [35]:
import six
import joblib
import sys
sys.modules['sklearn.externals.six'] = six
sys.modules['sklearn.externals.joblib'] = joblib
import pmdarima as pm
from pmdarima import auto_arima

In [36]:
def arima_model(df):
    """
    df- dataframe
    function is a gridsearch to get optimal p,d,qs and lowest for AIC for model.

    """
    autoarima_model = auto_arima(df, start_p = 0, start_q = 1, 
                              test='adf',             # use adftest to find optimal 'd'
                              max_p = 3, max_q = 3,   # maximum p and q
                              m = 12,                  #frequency of series 
                              d = None,               # let model determine 'd', was 1
                              seasonal = True, 
                              start_P=0, D=1, trace = False, #start  #trace= True
                              error_action ='ignore',   # we don't want to know if an order does not work 
                              suppress_warnings = True,  # we don't want convergence warnings 
                              stepwise = True)           # set to stepwise  
    
    #print('\n')
    #display(autoarima_model.summary())
    
    return autoarima_model


In [37]:
stepwise_fit = arima_model(train_brk[11226])

In [38]:
stepwise_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  214
Model:             SARIMAX(1, 1, 1)x(2, 1, 0, 12)   Log Likelihood               -2042.084
Date:                            Wed, 16 Dec 2020   AIC                           4096.169
Time:                                    20:16:18   BIC                           4115.989
Sample:                                         0   HQIC                          4104.189
                                            - 214                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    207.0293    282.514      0.733      0.464    -346.687     760.746
ar.L1          0.7237      0.053     13.635      0.000       0.620       0.828
ma.L1         -0.4512      0.053     -8.472      0.000      -0.556      -0.347
ar.S.L12      -0.2665      0.017    -15.377      0.000      -0.300      -0.233
ar.S.L24      -0.1027      0.015     -6.693      0.000      -0.133      -0.073
sigma2      3.838e+07      0.008   4.58e+09      0.000    3.84e+07    3.84e+07
===================================================================================
Ljung-Box (Q):                      184.31   Jarque-Bera (JB):               247.55
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.90   Skew:                             0.68
Prob(H) (two-sided):                  0.01   Kurtosis:                         8.27
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 8.87e+24. Standard errors may be unstable.
"""

### **Statsmodel Summary of Brooklyn Zipcodes**

In [ ]:
"""
for loop iterates through dataframe and gets the best fit parameters (p,d,qs, Seasonal p,d,qs) and lowest AICs 
  for each Brooklyn zipcode.
"""
arima_list = [['zipcode', 'pdq','seasonal_pdq','aic']] 
for col in zip_df.columns:
  zip_test_2 = arima_model(zip_df[col])
  arima_list.append([col,zip_test_2.order, zip_test_2.seasonal_order, zip_test_2.aic()])
#result   
output_df = pd.DataFrame(arima_list[1:],columns=arima_list[0]) 
output_df  

### **Dataframe of PDQs, Seasonal PDQs and AICs**

In [ ]:
output_df

## **SARIMA Model**

### **Fitting an ARIMA Time Series Model**
Using grid search, model identified the set of parameters that produces the best fitting model 
to time series data.  Then inputted the optimal parameter values into a new SARIMAX model.<br>    

Coef column above shows the importance of each feature and how each one impacts the time series patterns. 
The P>|z| provides the significance of each feature weight.<br>

Each weight has a p-value lower or close to 0.05, so it is reasonable to retain all of them in our model.<br>

model diagnostics-The purpose is to ensure that residuals remain uncorrelated, normally distributed having zero mean.  N(0,1)) is the standard notation for a normal distribution with mean 0 and standard deviation of 1).  This is a good indication that the residuals are normally distributed.<br>

In [ ]:
def fit_ARIMA(df, order=None, seasonal_order=None):
    """
    forecasting model
    """
    ARIMA_MODEL = sm.tsa.statespace.SARIMAX(df, 
                                        order=order, 
                                        seasonal_order=seasonal_order, 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

    # Fit the model and print results
    output = ARIMA_MODEL.fit()

    #display / no tables 1
    print(output.summary().tables[1])
    
    print('\n')
    print('MODEL DIAGNOSTICS')
    
    output.plot_diagnostics(figsize=(15, 18));
    
    return output

### **Validating the Model**

## **One-Step Ahead Forecasting**

In [ ]:
output_df

In [ ]:
train_brk[11238].head()

In [ ]:
train_brk[11238].tail()

In [ ]:
test_brk[11238][[0,-1]]

In [ ]:
"""
In order to validate model, I started by comparing predicted values to real values of the time series, which will help us understand the accuracy of 
    our forecasts

get_prediction() and .conf_int() methods allow us to obtain the values and associated confidence intervals for forecasts of the time series.
Get the confidence intervals for all predictions

"""
current_zip = 11218
zip_params= output_df[output_df['zipcode']==current_zip]
zip_params.pdq.values[0]
zip_params.seasonal_pdq.values[0]

output_sarima = fit_ARIMA(zip_df[current_zip],order=zip_params.pdq.values[0], seasonal_order= zip_params.seasonal_pdq.values[0] )
# Get dynamic predictions with confidence intervals as above 

pred = output_sarima.get_prediction(start=pd.to_datetime('2014-01-01'), dynamic=False)
pred_conf = pred.conf_int()

In [ ]:
"""
plot the real and forecasted values of the time series to assess how well model did
Plot the confidence intervals overlapping the predicted values
Plot real vs predicted values along with confidence interval
The forecasts align with the true values as seen above, with overall increase trend.
"""
plt.figure(figsize=(12,5))
# Plot observed values
ax = train_brk[11218]['1996':].plot(label='observed')
test_brk[11218]['1996':].plot(label='Test')
# Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=0.5)

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Sale Price')
plt.legend()
#confidence interval

In [ ]:
"""
check for the accuracy of the prediction using RMSE (Mean Squared Error). 
This will provide us with the average error of prediction

Model was able to forecast the average daily real estate sales in the test set within 8,068.75 of the real sales. 
  sales range from around 1,003,700.00 to 2,,202,400.00.     
"""
# Get the real and predicted values
forecasted_11238 = pred.predicted_mean
truth_1128 =test_brk[11218]['1996':]

# Compute the root mean square error
mse = ((forecasted_11238 - truth_1128) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
#np.sqrt(np.mean((predictions-targets)**2))
rmse = np.sqrt(np.mean((forecasted_11238 - truth_1128) ** 2))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))

In [ ]:
train_brk[zipcode].describe().round(3)

In [ ]:
zip_params = output_df[output_df['zipcode']==11218]
zip_params

In [ ]:
zip_params['pdq']

In [ ]:
zip_params.pdq.values[0]

In [ ]:
zip_params.seasonal_pdq.values[0]

In [ ]:
"""
we only use information from the time series up to a certain point, 
  and after that, forecasts are generated using values from previous forecasted time points.

pred_dynamic = output.get_prediction(start=pd.to_datetime('1998-01-01'), dynamic=True, full_results=True)
pred_dynamic_conf = pred_dynamic.conf_int()

calculation for predictions post 2014.
"""
current_zip = 11218
zip_params = output_df[output_df['zipcode']==current_zip]
zip_params.pdq.values[0]
zip_params.seasonal_pdq.values[0]

output_sarima = fit_ARIMA(zip_df[current_zip],order=zip_params.pdq.values[0] ,seasonal_order= zip_params.seasonal_pdq.values[0] )
# Get dynamic predictions with confidence intervals as above 
pred_dynamic = output_sarima.get_prediction(start=pd.to_datetime('2014-01-01'), dynamic=True,full_results=True)
pred_dynamic_conf = pred_dynamic.conf_int()

In [ ]:
"""
Plotting the observed and forecasted values of the time series, we see that the overall forecasts are accurate even when using dynamic forecasts. 
All forecasted values (yellow line) match pretty closely to the ground truth (blue line), 
  and are well within the confidence intervals of our forecast.
  
before used 2004
pred_dynamic = output_sarima.get_prediction(start=pd.to_datetime('2014-01-01'), dynamic=True,full_results=True)
pred_dynamic_conf = pred_dynamic.conf_int()
"""

def prediction_vis(pred_dynamic,pred_dynamic_conf, y):
  # Plot the dynamic forecast with confidence intervals.
  plt.figure(figsize=(12,5))
  # Plot observed values
  ax = y.plot(label='Observed')

  # Plot predicted values
  pred_dynamic.predicted_mean.plot(ax=ax, label='Dynamic Forecast', alpha=0.9)

  # Plot the range for confidence intervals
  ax.fill_between(pred_dynamic_conf.index,
                  pred_dynamic_conf.iloc[:, 0],
                  pred_dynamic_conf.iloc[:, 1], color='g', alpha=0.5)

  # Set axes labels
  ax.set_xlabel('Date')
  ax.set_ylabel('Sale Price')
  plt.legend()

  return ax

In [ ]:
prediction_visual = prediction_vis(pred_dynamic,pred_dynamic_conf,train_brk[current_zip])
prediction_visual

In [ ]:
# Get the real and predicted values
forecast_11238 = pred_dynamic.predicted_mean
truth_11238 = train_brk[current_zip]#['1996':]

# Compute the mean square error
mse = ((forecast_11238 - truth_11238) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
#np.sqrt(np.mean((predictions-targets)**2))
rmse = np.sqrt(np.mean((forecast_11238 - truth_11238) ** 2))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(rmse, 2)))

### **Return on Investment DataFrame**

In [ ]:
"""
.get_forecast() method of our time series output can compute forecasted values for a specified number of steps ahead.

prediction = output.get_forecast(steps=500)

# Get confidence intervals of forecasts
pred_conf = prediction.conf_int()

pred_dynamic = output_sarima.get_prediction(start=pd.to_datetime('2014-01-01'), dynamic=True,full_results=True)
pred_dynamic_conf = pred_dynamic.conf_int()
"""
# Get forecast --- steps ahead in future
prediction = output_sarima.get_forecast(steps=36, dynamic=True)
prediction.predicted_mean

# Get confidence intervals of forecasts
predict_conf = prediction.conf_int()


In [ ]:
#Ben
steps = 36
# Get forecast --- steps ahead in future
prediction_object = output_sarima.get_forecast(steps=steps, dynamic=True)

In [ ]:
def my_function(prediction_object, zip):
  """
  function gets ROI for 1 zipcode 
  """
  df_Summary = pd.concat([pd.DataFrame({f'Predicted_Mean {zip}':prediction_object.predicted_mean}), prediction_object.conf_int()],axis = 1)
  my_sample = df_Summary.iloc[[0, -1]].round(3)

  return my_sample

In [ ]:
my_output = my_function(prediction_object, zip='11218')
my_output

In [ ]:
"""
​	  
ROI= 
Cost of Investment
Current Value of Investment−Cost of Investment
​	 
"""
def my_roi(cost, current):
  #function to calculate ROI 
  return (current - cost) / cost


In [ ]:
cost = my_output.iloc[0,0]
current = my_output.iloc[-1,0]

In [ ]:

current_lower = my_output.iloc[-1,1]
current_upper = my_output.iloc[-1,2]

In [ ]:
#upper lower end
roi_dic = {}

cost = my_output.iloc[0,0]
current = my_output.iloc[-1,0]
current_lower = my_output.iloc[-1,1]
current_upper = my_output.iloc[-1,2]


my_roi(cost, current)
roi_dic['roi'] = my_roi(cost, current)
roi_dic['roi_lower'] = my_roi(cost, current_lower)
roi_dic['roi_upper'] = my_roi(cost, current_upper)

roi_dic

In [ ]:
zip_rois={}
steps = 36

#def zipcode_roi(output_df,):
for zipcode in output_df['zipcode'].unique():
  pdq = output_df.loc[ output_df['zipcode']==zipcode, 'pdq'].iloc[0] 
  seasonal = output_df.loc[ output_df['zipcode']==zipcode, 'seasonal_pdq'].iloc[0] 
  df_ts = zip_df[zipcode]


  output_sarima = fit_ARIMA(df_ts, order=pdq, seasonal_order=seasonal)
  prediction_object = output_sarima.get_forecast(steps=steps, dynamic=True)
  my_output = my_function(prediction_object, zip=zipcode)
  
  roi_dic = {}

  cost = my_output.iloc[0,0]
  current = my_output.iloc[-1,0]
  current_lower = my_output.iloc[-1,1]
  current_upper = my_output.iloc[-1,2]

  my_roi(cost, current)
  roi_dic['roi'] = my_roi(cost, current)
  roi_dic['roi_lower'] = my_roi(cost, current_lower)
  roi_dic['roi_upper'] = my_roi(cost, current_upper)

  zip_rois[zipcode] = pd.Series(roi_dic)
ROI = pd.DataFrame(zip_rois)

In [ ]:
roi_df = ROI.T 
roi_df.reset_index(inplace=True)
roi_df.rename(columns={'index':'zipcode'}, inplace=True)
roi_df.style.background_gradient()

In [ ]:
#changed column names
roi_chart = ROI.T
roi_chart.reset_index(inplace=True)
roi_chart.rename(columns={'index':'zipcode', 'roi_lower': '1_yr_roi','roi_upper':'3_yr_roi'}, inplace=True) 
roi_chart.style.background_gradient()

#### **ROI Mean Chart**

In [ ]:
roi_chart_1 = roi_chart.sort_values(by=['roi'],ascending=False)
#roi_chart_1 = roi_chart_1.round(3)
roi_chart_1.style.background_gradient()

#### **ROI 1st Year**

In [ ]:
roi_df_lower = roi_df.sort_values(by=['roi_lower'],ascending=False) 
roi_df_lower.style.background_gradient()
#roi_df_lower

#### **ROI 3rd Year**

In [ ]:
roi_df_upper = roi_df.sort_values(by=['roi_upper'],ascending=False) 
roi_df_upper.style.background_gradient()

## **Dynamic Forecasting**
Forecasting begins 4-1-2018

In [ ]:
def forecast_function(output_df, current_zip=None,steps=None):
  
  # roi_t[roi_t[roi_t.name]== current_zip]
  # print('\n')
  zip_params = output_df[output_df['zipcode']==current_zip]
  zip_params.pdq.values[0]
  zip_params.seasonal_pdq.values[0]

  #steps = 36
  output_sar = fit_ARIMA(zip_df[current_zip], order=zip_params.pdq.values[0], seasonal_order=zip_params.seasonal_pdq.values[0])
  prediction = output_sar.get_forecast(steps=steps, dynamic=True)
  prediction.predicted_mean

  # Get confidence intervals of forecasts
  predict_conf = prediction.conf_int()

  return prediction, predict_conf, current_zip


In [ ]:

def forecast_visual(prediction,predict_conf, y, figsize=None):
  """
  prediction-statsmodel object
  predict_conf- pd Dataframe
  """
  print(roi_df[roi_df['zipcode']== current_zip])
  print('\n')
  # Plot future predictions with confidence intervals
  fig,ax = plt.subplots(figsize=figsize)
  ax = y.plot(label='Observed') #(10, 8))
  prediction.predicted_mean.plot(ax=ax, label='Future Forecast')
  ax.fill_between(predict_conf.index,
                  predict_conf.iloc[:, 0],
                  predict_conf.iloc[:, 1], color='k', alpha=0.25)
  
  ax.axvline(prediction.predicted_mean.index[12])

  label_font = {'weight':'bold','size':18}
  ax.set_xlabel('Date',fontdict=label_font)
  ax.set_ylabel('Home Prices',fontdict=label_font)
  ax.set_title(f'Price Forecast for Zipcode: {y.name} /{steps} Months ',fontdict=label_font)

  ax.legend(loc="upper left")

  return ax





### **Zipcode: 11210**
Zipcode 11226 ranks in 1st place for ROI.  The ROI will be 60% on average.<br>
If an investment is made for a return on the lower end the return will be 5.27.<br> 
If a ninvestment is made for a return on the upper end the return will be 86.31.  This will take you into year 2021 and after.<br>
Either way it's a good return on the investment.

In [ ]:
prediction, predict_conf, current_zip =  forecast_function(output_df, current_zip=11210, steps=36)

In [ ]:
test_brk[11210].plot()

In [ ]:
test_brk[11210].describe()

In [ ]:
forecast_visual_zip = forecast_visual(prediction,predict_conf,test_brk[current_zip], figsize=(12,8))
forecast_visual_zip

### **Zipcode: 11233**

In [ ]:
prediction, predict_conf, current_zip =  forecast_function(output_df, current_zip=11233,steps=36)

In [ ]:
test_brk[11233].describe()

In [ ]:
test_brk[11233].plot()

In [ ]:
forecast_visual_zip = forecast_visual(prediction,predict_conf,test_brk[current_zip], figsize=(12,8))
forecast_visual_zip

### **Zipcode: 11218**

In [ ]:
prediction, predict_conf, current_zip =  forecast_function(output_df, current_zip=11218,steps=36)

In [ ]:
test_brk[11218].describe().round(3)

In [ ]:
test_brk[11218].plot()

In [ ]:
forecast_visual_zip = forecast_visual(prediction,predict_conf,test_brk[current_zip], figsize=(12,8))
forecast_visual_zip

### **Zipcode: 11230**

In [ ]:
prediction, predict_conf, current_zip =  forecast_function(output_df, current_zip=11230,steps=36)

In [ ]:
test_brk[11230].describe().round(3)

In [ ]:
test_brk[current_zip].plot()

In [ ]:
forecast_visual_zip = forecast_visual(prediction,predict_conf,test_brk[current_zip], figsize=(12,8))
forecast_visual_zip

### **Zipcode: 11212**

In [ ]:
prediction, predict_conf, current_zip =  forecast_function(output_df, current_zip=11212,steps=36)

In [ ]:
test_brk[11212].describe().round(3)

In [ ]:
test_brk[current_zip].plot()

In [ ]:
#11212	0.371624	-0.027713	0.770960
forecast_visual_zip = forecast_visual(prediction,predict_conf,test_brk[current_zip], figsize=(12,8))
forecast_visual_zip

## ***Stationarity***

### **Zipcode: 11226**

In [ ]:
zip_df[11226].plot()

In [ ]:
def test_stationarity_1(timeseries, window):
    
    #Defining rolling statistics
    rolmean = timeseries.rolling(window=window).mean()
    rolstd = timeseries.rolling(window=window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries.iloc[window:], color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='upper left')
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend(bbox_to_anchor=(1.05,1),loc='upper left')
    plt.show()
    

In [ ]:
test_stationarity_1(zip_df,12)

In [ ]:
#Not mine

def dickey_fuller_test_ind_zip(zip_code):
    dftest = adfuller(zip_code)

    # Extract and display test results in a user friendly manner
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dftest)

    print ('Results of Dickey-Fuller Test:')

    return dfoutput

In [ ]:
new_dic = {}
for col in zip_df.columns:
  zip_test = dickey_fuller_test_ind_zip(zip_df[col])
  new_dic[col] = zip_test

In [ ]:
new_dic[11226]

In [ ]:

def dickey_fuller_test_zipcodes(df):
    for col in df.columns:
        dftest = adfuller(df[col])
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        print ('Results of Dickey-Fuller Test:')
        
        print(dfoutput) 
        #print(dftest)
        print ('\n')         

In [ ]:
dickey_fuller_test_zipcodes(zip_df)

In [ ]:
X_1 = zip_df.copy()

In [ ]:
def stationary_test(df):
    rolling_mean = df.rolling(window=12).mean()
    rolling_std = df.rolling(window=12).std()

    plt.plot(df,color='blue',label='orignal')
    plt.plot(rolling_mean, color='red',label='Rolling Mean')
    plt.plot(rolling_std, color='green',label='Rolling STD')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Rolling Standard Deviation')
    #plt.show()
    result = adfuller(df)
    print('ADF statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{} : {}'.format(key,value))
        
    names = ['Test Statistic','p-value','#Lags Used','# of Observations Used']
    res  = dict(zip(names,result[:4]))
    res['Stationary Results'] = res['p-value']<.05
    
    return pd.DataFrame(res,index=['AD Fuller Results'])    

In [ ]:
stationary_test(zip_df[11226])

###  Zipcode:  11238

In [ ]:
#brooklyn_zips[11226]

In [ ]:
#stationary_test(zip_df[11238])

### Zipcode:  11215

In [ ]:
stationary_test(zip_df[11226])

### Removing Trend
#### Log-Transformation (np.log)

In [ ]:
## Log Transform
ts3 = np.log(zip_df[11226])
#ts3.plot()
stationary_test(ts3)

#### Differencing

In [ ]:
"""
#subtracts the ts 1 step forward from itself. Good way of eliminting trend

#below ts centered around 0
#we achieved stationarity
#eliminating day-to-day patterns
"""
ts0 = zip_df[11226].diff().dropna()
#ts0.plot()

stationary_test(ts0)

#### Subtract Rolling Mean 

In [ ]:
## Subtract Rolling mean
ts2 = (zip_df[11226] - zip_df[11226].rolling(3).mean()).dropna()
#ts2.plot()
stationary_test(ts2)

#### Subtract Exponentially-Weighted Mean 

In [ ]:
## Subtract Exponentially Weight Mean Rolling mean
ts4 = (zip_df[11226] - zip_df[11226].ewm(halflife=7).mean()).dropna()
#ts4.plot()
stationary_test(ts4)

#### Seasonal Decomposition 

In [ ]:
"""
it will identfies patterns, trends in ts and separate them into 3 new ts.
residual-what was left over after removing all the other components.
"""
from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(zip_df[11226])#,model='mul')
decomp.plot();

In [ ]:
## Get ADFuller Results for seasonal component
stationary_test(decomp.seasonal)

In [ ]:
## Get ADFuller Results for trend component
stationary_test(decomp.trend.dropna())

In [ ]:
## Get ADFuller Results for resid component
stationary_test(decomp.resid.dropna())

In [ ]:
decomp.resid.dropna()

## **RNN**

In [ ]:
df_rnn = zip_df[[11238]]

In [ ]:
df_rnn.head()

In [ ]:
df_rnn.plot()

In [ ]:
len(df_rnn)

In [ ]:
265-12

In [ ]:
"""
x_train= x_train.reshape(-1, 1)
y_train= y_train.reshape(-1, 1)
x_test = x_test.reshape(-1, 1)
"""
train = df_rnn.iloc[:253]
test = df_rnn.iloc[253:]
#test = test.reshape(1, -1)
#train= train.reshape(-1, 1)

In [ ]:
test

In [ ]:
len(test)

In [ ]:
train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
n_input = 2
n_features = 1 #smaller batch sizes lead to better training

generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
scaled_train[:5]

In [ ]:
len(scaled_train)

In [ ]:
"""

253 - n_input(2)

"""
len(generator)

In [ ]:
#create model and fit it to the generator object
from keras.models import Sequential
from keras.layers import Dense  #for final output later
from keras.layers import LSTM #long short term memory

In [ ]:
n_input = 12 #look at full year of data or 12 months before predicting 13th month
n_features = 1 #smaller batch sizes lead to better training
               #how many columns you have. WE have 1 column which is time stamp for y

train_generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
model = Sequential()

model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
#need to aggregate all the neurons to sngle prediciton
model.add(Dense(1)) #added single dense neuron which will directly output our prediction
model.compile(optimizer='adam', loss='mse')

In [ ]:
"""
may want to play around w/number of neurons on LSTM layer
"""
model.summary()

In [ ]:
"""
fit tou our training generator
more epochs you use hte longer it's going to take to train
1 epoch is a single entire run through of training data

We get significant reduciton over 1st couple of epochs then around 15 start seeing convergence

"""
model.fit_generator(train_generator, epochs=25)

In [ ]:
model.history.history.keys()

In [ ]:
plt.plot(range(len(model.history.history['loss'])),model.history.history['loss']);

In [ ]:
"""
evalute on the test data
create an evlauation batch
our network trains 1 step ahead

our network is 12 network steps 
    then predict step 13
    
need last 12 points of training data inorder to predict pt. 1 of test data 

these are last 12 points of training set
"""
first_eval_batch = scaled_train[-12:]
first_eval_batch

In [ ]:
"""
it now has 3 brackets at the top
"""
first_eval_batch = first_eval_batch.reshape((1,n_input,n_features))
first_eval_batch

In [ ]:
"""
call model on first_eval_batch
gives array prediciton
means given these 12 points of training data it predicts taht below should be 1st point of test data set
"""
model.predict(first_eval_batch)

In [ ]:
scaled_test

In [ ]:
"""
not just predict 1st point in test set but the entire test set
how to forecast into the future
Forecast using RNN model
"""
#hold predicitons
test_predictions = []
#last n_input points from training set
first_eval_batch = scaled_train[-n_input:] 
#reshape to format of RNN wants, (same format as Timeseriesgenerator 
current_batch = first_eval_batch.reshape((1,n_input,n_features))

#hoe far into the futrue will I forecast: length of test set
for i in range(len(test)):
    #1time step ahead of historical 12 points
    current_pred = model.predict(current_batch)[0] #0 is for formatting 
    test_predictions.append(current_pred)
    
    #update current batch to include prediciton
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
true_predictions

In [ ]:
test['Predictions'] = true_predictions

In [ ]:
test

### **RNN Plot / Sales v Predicted Values**

In [ ]:
"""
sales v predicted values
"""
test.plot(figsize=(12,5));

### **Recommendations**
Top Five zipcodes to invest in w/mean ROI:<br>
11210  (60%)<br>
11233  (42.5%)<br>
11218  (37.8%)<br>
11230  (37.8%)<br>
11212  (37%)<br>

Top Five zipcodes to invest in ROI 1st yr.:<br>
11238  (19%)<br>
11217  (15%)<br>
11220  (11%)<br>
11230  (9.8%)<br>
11224  (8.7%)<br>

Top Five zipcodes to invest in ROI 3rd yr.:<br>
11210  (489%)<br>
11221  (192%)<br>
11218  (90%)<br>
11233  (79%)<br>
11212  (77%)<br>